# Gradio Demo: chatbot_multimodal

In [ ]:
!pip install -q gradio plotly

In [ ]:
# Downloading files from the demo repo
import os
os.mkdir('files')
!wget -q -O files/avatar.png https://github.com/gradio-app/gradio/raw/main/demo/chatbot_multimodal/files/avatar.png
!wget -q -O files/cantina.wav https://github.com/gradio-app/gradio/raw/main/demo/chatbot_multimodal/files/cantina.wav
!wget -q https://github.com/gradio-app/gradio/raw/main/demo/chatbot_multimodal/messages_testcase.py

In [ ]:
import gradio as gr
import plotly.express as px
import os
# Chatbot demo with multimodal input (text, markdown, LaTeX, code blocks, image, audio, & video). Plus shows support for streaming text.

def random_plot():
    df = px.data.iris()
    fig = px.scatter(df, x="sepal_width", y="sepal_length", color="species",
                    size='petal_length', hover_data=['petal_width'])
    return fig

def print_like_dislike(x: gr.LikeData):
    print(x.index, x.value, x.liked)

def add_message(history, message):
    for x in message["files"]:
        history.append(((x,), None))
    if message["text"] is not None:
        history.append((message["text"], None))
    return history, gr.MultimodalTextbox(value=None, interactive=False)

def add_example_message(x: gr.SelectData, history):
    if x.value["text"] is not None:
        history.append((x.value["text"], None))
    if "files" in x.value:
        if isinstance(x.value["files"], list):
            for file in x.value["files"]:
                history.append((file, None))
        else:
            history.append((x.value["files"], None))
    return history

def bot(history):
    history[-1][1] = "Cool!"
    return history

fig = random_plot()

with gr.Blocks(fill_height=True) as demo:
    chatbot = gr.Chatbot(
        elem_id="chatbot",
        bubble_full_width=False,
        scale=1,
        placeholder='<h1 style="font-weight: bold; color: #FF6B6B; text-align: center; font-size: 48px; font-family: Arial, sans-serif;">Welcome to Gradio!</h1>',
        examples=[{"text": "Try this example with this audio.", "files": {"path": "files/cantina.wav"}}, {"text": "Try this example with this image.", "files": {"path": "files/avatar.png"}}, {"text": "This is just text, no files!"}, {"text": "Try this example with this image.", "files": [{"path": "files/avatar.png"}, {"path": "files/avatar.png"}]}],
    )

    chat_input = gr.MultimodalTextbox(interactive=True,
                                      file_count="multiple",
                                      placeholder="Enter message or upload file...", show_label=False)

    chat_msg = chat_input.submit(add_message, [chatbot, chat_input], [chatbot, chat_input])
    bot_msg = chat_msg.then(bot, chatbot, chatbot, api_name="bot_response")
    bot_msg.then(lambda: gr.MultimodalTextbox(interactive=True), None, [chat_input])

    chatbot.like(print_like_dislike, None, None)
    chatbot.example_select(add_example_message, [chatbot], [chatbot])

if __name__ == "__main__":
    demo.launch()
